In [21]:
import os
import shutil

from azureml.core.workspace import Workspace
from azureml.core import Experiment
from azureml.core.environment import Environment
from azureml.train.automl.run import AutoMLRun as run
from azureml.core.model import Model

from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
from azureml.core.conda_dependencies import CondaDependencies
from azureml.train.dnn import PyTorch

print("Azure ML SDK Version: ", azureml.core.VERSION)

Azure ML SDK Version:  1.0.85


In [22]:
# ws = Workspace.from_config(path="./config.json")
ws = Workspace.from_config()

model=Model(ws, "model_deploy_synbiolic")
model.download(target_dir=os.getcwd(), exist_ok=True)
file_path=os.path.join(os.getcwd(), "modeldir.ckpt")
os.stat(file_path)

os.stat_result(st_mode=33279, st_ino=8441, st_dev=46, st_nlink=1, st_uid=0, st_gid=0, st_size=90602068, st_atime=1584504056, st_mtime=1584504056, st_ctime=1584580293)

In [3]:
project_folder = './train-main'
experiment_name = 'synbio7'
experiment = Experiment(ws, name=experiment_name)
cluster_name = "gpu-compute"
compute_target = ComputeTarget(workspace=ws, name=cluster_name)
print(f'Found existing compute target: {compute_target}')

Found existing compute target: AmlCompute(workspace=Workspace.create(name='ml-main', subscription_id='f1be6293-6d1c-4e77-bb06-b2083e9aeb20', resource_group='synbiolic-main'), name=gpu-compute, id=/subscriptions/f1be6293-6d1c-4e77-bb06-b2083e9aeb20/resourceGroups/synbiolic-main/providers/Microsoft.MachineLearningServices/workspaces/ml-main/computes/gpu-compute, type=AmlCompute, provisioning_state=Succeeded, location=eastus, tags=None)


In [6]:
model = Model.register(workspace=ws,
                      model_path="../train-main/outputs/modeldir.ckpt",
                      model_name="model_deploy_synbiolic", 
                      description="runn")

Registering model model_deploy_synbiolic


In [7]:
!pip install inference_schema

  Created wheel for wrapt: filename=wrapt-1.11.1-cp36-cp36m-linux_x86_64.whl size=66568 sha256=2e7c0954ac64f38ca3630a579a9d92769f468d2d42c9a995a88c300151375b91
  Stored in directory: /home/azureuser/.cache/pip/wheels/89/67/41/63cbf0f6ac0a6156588b9587be4db5565f8c6d8ccef98202fc
Successfully built wrapt
ERROR: astroid 2.3.1 has requirement six==1.12, but you'll have six 1.14.0 which is incompatible.
  Found existing installation: wrapt 1.11.2
    Uninstalling wrapt-1.11.2:
      Successfully uninstalled wrapt-1.11.2


In [8]:
from inference_schema.schema_decorators import input_schema, output_schema
from inference_schema.parameter_types.standard_py_parameter_type import StandardPythonParameterType
import os
import numpy as np
from sklearn.externals import joblib

def init():
    global model
    model_path = os.path.join(os.getenv("AZUREML_MODEL_DIR"), "modeldir.ckpt")
    model = joblib.load(model_path)

input_sample = 1

def run(data):
    try:
        print(model)
        result = model.predict(data)
        return result
    except Exception as e:
        error = str(e)
        return error

In [10]:
a = input_schema("data", StandardPythonParameterType(input_sample))
print(input_schema("data", StandardPythonParameterType(input_sample)))


<function input_schema.<locals>.decorator_input at 0x7fbb5967c158>


In [26]:
!ls
from utils.py import read_smi_file

data_folder = os.path.join(os.getcwd(), "data")
#test = 

condadep.yml	     Descriptors.ipynb	pytorchestimator.ipynb
copy2_trainer.ipynb  modeldeploy.ipynb
copy_trainer.ipynb   modeldir.ckpt


ModuleNotFoundError: No module named 'utils'